# 1. Import and Install Dependencies

In [1]:
!pip install tensorflow==2.12.0 opencv-python mediapipe sklearn matplotlib

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import winsound

# 2. Keypoints using MP Holistic

In [3]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [4]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [5]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [6]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

# 3. Extract Keypoint Values

In [7]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

# 4. Setup Folders for Collection

In [8]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
actions = np.array(['BharathaNatyam', 'Kathak', 'Bangra'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 0

In [9]:
for action in actions: 
    #dirmax = np.max(np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int))
    #for sequence in range(1,no_sequences+1):
     for sequence in range(no_sequences):
        try: 
            #os.makedirs(os.path.join(DATA_PATH, action, str(dirmax+sequence)))
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# 5. Collect Keypoint Values for Training and Testing

In [11]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(start_folder, start_folder+no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    winsound.Beep(2500,500)
                    cv2.waitKey(2000)
                    winsound.Beep(1000,1000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [12]:
cap.release()
cv2.destroyAllWindows()

# 6. Preprocess Data and Create Labels and Features

In [13]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [14]:
label_map = {label:num for num, label in enumerate(actions)}

In [15]:
label_map

{'pose1': 0, 'pose2': 1, 'pose3': 2}

In [16]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [17]:
# Ok to skip
np.array(sequences).shape

(90, 30, 1662)

In [18]:
# Ok to skip
np.array(labels).shape

(90,)

In [19]:
# Ok to skip
X = np.array(sequences)

In [20]:
# Ok to skip
X.shape

(90, 30, 1662)

In [21]:
# Ok to skip
y = to_categorical(labels).astype(int)

In [22]:
# Ok to skip
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [23]:
# Ok to skip
y_test.shape

(5, 3)

# 7. Build and Train LSTM Neural Network

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [11]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [26]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [27]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [28]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
3/3 [==============================] - 4s 51ms/step - loss: 5.5248 - categorical_accuracy: 0.3647
Epoch 2/2000
3/3 [==============================] - 0s 51ms/step - loss: 22.8344 - categorical_accuracy: 0.3529
Epoch 3/2000
3/3 [==============================] - 0s 50ms/step - loss: 66.4271 - categorical_accuracy: 0.3882
Epoch 4/2000
3/3 [==============================] - 0s 50ms/step - loss: 75.1637 - categorical_accuracy: 0.2824
Epoch 5/2000
3/3 [==============================] - 0s 50ms/step - loss: 156.2271 - categorical_accuracy: 0.3412
Epoch 6/2000
3/3 [==============================] - 0s 50ms/step - loss: 11.3849 - categorical_accuracy: 0.3882
Epoch 7/2000
3/3 [==============================] - 0s 51ms/step - loss: 19.9161 - categorical_accuracy: 0.3412
Epoch 8/2000
3/3 [==============================] - 0s 50ms/step - loss: 3.8393 - categorical_accuracy: 0.3765
Epoch 9/2000
3/3 [==============================] - 0s 51ms/step - loss: 5.8597 - categorical_accuracy: 0

3/3 [==============================] - 0s 55ms/step - loss: 0.4027 - categorical_accuracy: 0.8353
Epoch 147/2000
3/3 [==============================] - 0s 56ms/step - loss: 0.3712 - categorical_accuracy: 0.8471
Epoch 148/2000
3/3 [==============================] - 0s 55ms/step - loss: 0.3848 - categorical_accuracy: 0.8118
Epoch 149/2000
3/3 [==============================] - 0s 53ms/step - loss: 0.4607 - categorical_accuracy: 0.7765
Epoch 150/2000
3/3 [==============================] - 0s 56ms/step - loss: 0.3945 - categorical_accuracy: 0.8000
Epoch 151/2000
3/3 [==============================] - 0s 55ms/step - loss: 0.4339 - categorical_accuracy: 0.7529
Epoch 152/2000
3/3 [==============================] - 0s 54ms/step - loss: 0.3729 - categorical_accuracy: 0.8588
Epoch 153/2000
3/3 [==============================] - 0s 54ms/step - loss: 0.3702 - categorical_accuracy: 0.8471
Epoch 154/2000
3/3 [==============================] - 0s 54ms/step - loss: 0.3813 - categorical_accuracy: 0.835

3/3 [==============================] - 0s 59ms/step - loss: 0.5641 - categorical_accuracy: 0.6941
Epoch 291/2000
3/3 [==============================] - 0s 57ms/step - loss: 0.5642 - categorical_accuracy: 0.7059
Epoch 292/2000
3/3 [==============================] - 0s 58ms/step - loss: 0.5617 - categorical_accuracy: 0.7294
Epoch 293/2000
3/3 [==============================] - 0s 58ms/step - loss: 0.5634 - categorical_accuracy: 0.6706
Epoch 294/2000
3/3 [==============================] - 0s 60ms/step - loss: 0.5620 - categorical_accuracy: 0.6471
Epoch 295/2000
3/3 [==============================] - 0s 60ms/step - loss: 0.5589 - categorical_accuracy: 0.7294
Epoch 296/2000
3/3 [==============================] - 0s 59ms/step - loss: 0.5560 - categorical_accuracy: 0.7059
Epoch 297/2000
3/3 [==============================] - 0s 58ms/step - loss: 0.5580 - categorical_accuracy: 0.6000
Epoch 298/2000
3/3 [==============================] - 0s 58ms/step - loss: 0.5570 - categorical_accuracy: 0.635

3/3 [==============================] - 0s 61ms/step - loss: 1.0186 - categorical_accuracy: 0.5882
Epoch 435/2000
3/3 [==============================] - 0s 63ms/step - loss: 0.6964 - categorical_accuracy: 0.6353
Epoch 436/2000
3/3 [==============================] - 0s 61ms/step - loss: 0.6201 - categorical_accuracy: 0.6471
Epoch 437/2000
3/3 [==============================] - 0s 61ms/step - loss: 0.6865 - categorical_accuracy: 0.6471
Epoch 438/2000
3/3 [==============================] - 0s 62ms/step - loss: 0.5875 - categorical_accuracy: 0.6000
Epoch 439/2000
3/3 [==============================] - 0s 61ms/step - loss: 0.5588 - categorical_accuracy: 0.7412
Epoch 440/2000
3/3 [==============================] - 0s 60ms/step - loss: 0.5874 - categorical_accuracy: 0.7294
Epoch 441/2000
3/3 [==============================] - 0s 60ms/step - loss: 0.5952 - categorical_accuracy: 0.6471
Epoch 442/2000
3/3 [==============================] - 0s 60ms/step - loss: 0.5611 - categorical_accuracy: 0.682

3/3 [==============================] - 0s 62ms/step - loss: 0.5051 - categorical_accuracy: 0.7176
Epoch 579/2000
3/3 [==============================] - 0s 63ms/step - loss: 0.5046 - categorical_accuracy: 0.6941
Epoch 580/2000
3/3 [==============================] - 0s 62ms/step - loss: 0.5093 - categorical_accuracy: 0.7529
Epoch 581/2000
3/3 [==============================] - 0s 63ms/step - loss: 0.4997 - categorical_accuracy: 0.8118
Epoch 582/2000
3/3 [==============================] - 0s 64ms/step - loss: 0.5164 - categorical_accuracy: 0.7882
Epoch 583/2000
3/3 [==============================] - 0s 63ms/step - loss: 0.5072 - categorical_accuracy: 0.7647
Epoch 584/2000
3/3 [==============================] - 0s 63ms/step - loss: 0.5094 - categorical_accuracy: 0.6588
Epoch 585/2000
3/3 [==============================] - 0s 63ms/step - loss: 0.5075 - categorical_accuracy: 0.7765
Epoch 586/2000
3/3 [==============================] - 0s 65ms/step - loss: 0.5024 - categorical_accuracy: 0.800

...

3/3 [==============================] - 0s 65ms/step - loss: 0.2848 - categorical_accuracy: 0.8941
Epoch 1659/2000
3/3 [==============================] - 0s 64ms/step - loss: 0.2632 - categorical_accuracy: 0.8824
Epoch 1660/2000
3/3 [==============================] - 0s 64ms/step - loss: 0.2862 - categorical_accuracy: 0.8824
Epoch 1661/2000
3/3 [==============================] - 0s 65ms/step - loss: 0.3236 - categorical_accuracy: 0.8471
Epoch 1662/2000
3/3 [==============================] - 0s 65ms/step - loss: 0.2707 - categorical_accuracy: 0.8706
Epoch 1663/2000
3/3 [==============================] - 0s 64ms/step - loss: 0.3423 - categorical_accuracy: 0.8706
Epoch 1664/2000
3/3 [==============================] - 0s 66ms/step - loss: 0.3885 - categorical_accuracy: 0.8824
Epoch 1665/2000
3/3 [==============================] - 0s 65ms/step - loss: 0.2875 - categorical_accuracy: 0.8941
Epoch 1666/2000
3/3 [==============================] - 0s 65ms/step - loss: 0.2852 - categorical_accurac

3/3 [==============================] - 0s 64ms/step - loss: 0.3237 - categorical_accuracy: 0.8353
Epoch 1731/2000
3/3 [==============================] - 0s 64ms/step - loss: 0.2912 - categorical_accuracy: 0.8824
Epoch 1732/2000
3/3 [==============================] - 0s 65ms/step - loss: 0.2787 - categorical_accuracy: 0.8824
Epoch 1733/2000
3/3 [==============================] - 0s 65ms/step - loss: 0.2820 - categorical_accuracy: 0.8941
Epoch 1734/2000
3/3 [==============================] - 0s 65ms/step - loss: 0.4128 - categorical_accuracy: 0.8588
Epoch 1735/2000
3/3 [==============================] - 0s 63ms/step - loss: 0.2979 - categorical_accuracy: 0.8941
Epoch 1736/2000
3/3 [==============================] - 0s 63ms/step - loss: 0.2865 - categorical_accuracy: 0.8824
Epoch 1737/2000
3/3 [==============================] - 0s 64ms/step - loss: 0.3079 - categorical_accuracy: 0.8235
Epoch 1738/2000
3/3 [==============================] - 0s 63ms/step - loss: 0.3787 - categorical_accurac

3/3 [==============================] - 0s 65ms/step - loss: 0.2537 - categorical_accuracy: 0.8824
Epoch 1803/2000
3/3 [==============================] - 0s 65ms/step - loss: 0.2564 - categorical_accuracy: 0.9059
Epoch 1804/2000
3/3 [==============================] - 0s 65ms/step - loss: 0.2279 - categorical_accuracy: 0.9059
Epoch 1805/2000
3/3 [==============================] - 0s 64ms/step - loss: 0.2375 - categorical_accuracy: 0.9059
Epoch 1806/2000
3/3 [==============================] - 0s 65ms/step - loss: 0.2990 - categorical_accuracy: 0.8588
Epoch 1807/2000
3/3 [==============================] - 0s 64ms/step - loss: 0.3815 - categorical_accuracy: 0.8471
Epoch 1808/2000
3/3 [==============================] - 0s 65ms/step - loss: 0.3053 - categorical_accuracy: 0.8706
Epoch 1809/2000
3/3 [==============================] - 0s 65ms/step - loss: 0.2514 - categorical_accuracy: 0.9059
Epoch 1810/2000
3/3 [==============================] - 0s 65ms/step - loss: 0.2653 - categorical_accurac

3/3 [==============================] - 0s 66ms/step - loss: 0.3327 - categorical_accuracy: 0.8824
Epoch 1875/2000
3/3 [==============================] - 0s 66ms/step - loss: 0.2584 - categorical_accuracy: 0.8941
Epoch 1876/2000
3/3 [==============================] - 0s 66ms/step - loss: 0.2648 - categorical_accuracy: 0.8941
Epoch 1877/2000
3/3 [==============================] - 0s 65ms/step - loss: 0.2472 - categorical_accuracy: 0.9176
Epoch 1878/2000
3/3 [==============================] - 0s 74ms/step - loss: 0.2452 - categorical_accuracy: 0.9294
Epoch 1879/2000
3/3 [==============================] - 0s 67ms/step - loss: 0.2073 - categorical_accuracy: 0.9176
Epoch 1880/2000
3/3 [==============================] - 0s 67ms/step - loss: 0.2480 - categorical_accuracy: 0.8824
Epoch 1881/2000
3/3 [==============================] - 0s 67ms/step - loss: 0.2419 - categorical_accuracy: 0.8941
Epoch 1882/2000
3/3 [==============================] - 0s 67ms/step - loss: 0.2426 - categorical_accurac

3/3 [==============================] - 0s 64ms/step - loss: 0.2134 - categorical_accuracy: 0.9412
Epoch 1947/2000
3/3 [==============================] - 0s 72ms/step - loss: 0.3046 - categorical_accuracy: 0.8706
Epoch 1948/2000
3/3 [==============================] - 0s 66ms/step - loss: 0.2435 - categorical_accuracy: 0.9059
Epoch 1949/2000
3/3 [==============================] - 0s 65ms/step - loss: 0.2063 - categorical_accuracy: 0.9412
Epoch 1950/2000
3/3 [==============================] - 0s 64ms/step - loss: 0.2528 - categorical_accuracy: 0.9294
Epoch 1951/2000
3/3 [==============================] - 0s 65ms/step - loss: 0.2097 - categorical_accuracy: 0.9294
Epoch 1952/2000
3/3 [==============================] - 0s 65ms/step - loss: 0.1942 - categorical_accuracy: 0.9294
Epoch 1953/2000
3/3 [==============================] - 0s 72ms/step - loss: 0.1879 - categorical_accuracy: 0.9294
Epoch 1954/2000
3/3 [==============================] - 0s 65ms/step - loss: 0.1941 - categorical_accurac

In [29]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 3)                 99        
                                                                 
Total params: 596,675
Trainable params: 596,675
Non-trai

# 8. Make Predictions

In [30]:
res = model.predict(X_test)

1/1 [==============================] - 0s 345ms/step


In [31]:
actions[np.argmax(res[4])]

'pose1'

In [32]:
actions[np.argmax(y_test[4])]

'pose1'

# 9. Save Weights

In [33]:
model.save('action.h5')

In [34]:
del model

In [12]:
# Once training is done, u do not have to train it again (no need to call the fit() finction). Just need to setup the model
# and compile it. then load the saved weights.
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [13]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [14]:
model.load_weights('action.h5')

# 10. Evaluation using Confusion Matrix and Accuracy

In [39]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [40]:
yhat = model.predict(X_test)

1/1 [==============================] - 0s 327ms/step


In [41]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [42]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[3, 0],
        [0, 2]],

       [[4, 0],
        [1, 0]],

       [[2, 1],
        [0, 2]]], dtype=int64)

In [43]:
accuracy_score(ytrue, yhat)

0.8

# 11. Test in Real Time

In [15]:
from scipy import stats

In [16]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [17]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, actions, image, colors))

NameError: name 'res' is not defined

<Figure size 1800x1800 with 0 Axes>

In [18]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

1/1 [==============================] - 0s 44ms/step
pose3
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 27ms/step
pose3
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 25ms/step
pose3
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 25ms/step
pose3
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 34ms/step
pose2
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 35ms/step
pose2
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 31ms/step
pose2
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 26ms/step
pose2
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 27ms/step
pose2
<class 'mediapipe.pyth

1/1 [==============================] - 0s 26ms/step
pose1
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 27ms/step
pose1
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 22ms/step
pose1
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 27ms/step
pose1
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 43ms/step
pose1
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 25ms/step
pose2
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 26ms/step
pose2
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 25ms/step
pose1
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 25ms/step
pose2
<class 'mediapipe.pyth

1/1 [==============================] - 0s 24ms/step
pose1
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 31ms/step
pose2
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 25ms/step
pose1
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 24ms/step
pose1
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 34ms/step
pose1
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 37ms/step
pose1
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 27ms/step
pose2
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 25ms/step
pose1
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 25ms/step
pose1
<class 'mediapipe.pyth